In [1]:
!pip install torch numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 33.3 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 48.2 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 170.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 161.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 63.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 45.5 MB/s  0:00:07m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 143.1 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 149.9 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 110.3 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 105.5 MB/s  0:00:020:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [1]:
"""
如何看代码
1 句子token
2 token后embedding
3 
"""

'\n如何看代码\n1 句子token\n2 token后embedding\n3 \n'

In [4]:
import math,torch
import torch.nn as nn
from torch.autograd import Variable #变量封装函数

In [ ]:
#数据token后，进行embedding
class Embedding(nn.Module):
    def __init__(self, d_model,vocab):
        super(Embedding,self).__init__()
        self.lut=nn.Embedding(vocab,d_model)
        self.d_model=d_model
    def forward(self,x):
        return self.lut(x)*math.sqrt(self.d_model)
'''
embedding = Embedding(10,30)
input=torch.tensor([[1,2,4,5],[5,2,5,6]],dtype=torch.long)
embedding(input)
'''

#position emb


tensor([[[ 1.3580, -2.2336, -0.3656, -0.6157, -2.4183, -2.3345,  2.4852,
           3.1115, -1.0474,  2.0119],
         [ 8.6090,  0.8809, -2.8904,  0.0165, -4.0032, -2.2990, -5.1315,
           5.9497, -1.0014, -0.3037],
         [ 6.7013, -0.9624,  3.4766,  6.0569,  1.4351, -0.9494, -1.9784,
           0.9103, -1.4205, -3.4482],
         [ 0.5417, -0.6769,  4.9958, -0.6726, -3.5746,  2.7277, -0.5045,
           3.4489,  5.1029, -5.5942]],

        [[ 0.5417, -0.6769,  4.9958, -0.6726, -3.5746,  2.7277, -0.5045,
           3.4489,  5.1029, -5.5942],
         [ 8.6090,  0.8809, -2.8904,  0.0165, -4.0032, -2.2990, -5.1315,
           5.9497, -1.0014, -0.3037],
         [ 0.5417, -0.6769,  4.9958, -0.6726, -3.5746,  2.7277, -0.5045,
           3.4489,  5.1029, -5.5942],
         [-1.7509, -2.4829, -1.3953,  2.1080,  2.4004,  6.0079, -5.1182,
          -0.3051, -2.3291, -0.8537]]], grad_fn=<MulBackward0>)